In [1]:
# import packages
import numpy as np # linear algebra
import pandas as pd 
import re
from nltk.corpus import stopwords
import nltk
from nltk.util import ngrams
stop = stopwords.words('english')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [2]:
# read data
true = pd.read_csv("True.csv")
fake = pd.read_csv("Fake.csv")
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [3]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
# add target value and merge df
fake['target'] = 0
true['target'] = 1
news = pd.concat([fake, true])
news = shuffle(news).reset_index(drop = True)

print(news.target.value_counts())

0    23481
1    21417
Name: target, dtype: int64


In [5]:
# def clean function of text
def clean(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub("[0-9]*",'', text)
    text = re.sub("\@", '', text.lower())
    text = re.sub(r'[^\w\s]', '', text).split()
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english')
    text = [wnl.lemmatize(word) for word in text if word not in stopwords]
    text = ' '.join(text)
    return text

In [6]:
# clean text
news['text'] = news['text'].apply(clean)

In [7]:
news.head()

,title,text,subject,date,target
0,Ugandan parliament passes law allowing Museven...,kampala reuters ugandan legislator voted overw...,worldnews,"December 20, 2017",1
1,Elite Nazi-allied Order From Hungary Claims Tr...,lili bayer larry cohleresses haaretzsebastian ...,Middle-east,"March 24, 2017",0
2,U.S. service member killed in Iraq IED blast: ...,washington reuters u service member coalition ...,worldnews,"October 2, 2017",1
3,Trump Just Threw A Colossal Hissy Fit On Twit...,donald trump maturity toddler certainly put fu...,News,"August 30, 2017",0
4,The “Safe Welfare State” Of Sweden Descends In...,swede apparently reached limit tolerance musli...,left-news,"Nov 14, 2015",0


In [8]:
# train with a basic model lr
x_train,x_test,y_train,y_test = train_test_split(news['text'], news.target, test_size=0.2, random_state=2020)

pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LogisticRegression())])

model = pipe.fit(x_train, y_train)
prediction = model.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))
print(confusion_matrix(y_test, prediction))
print(classification_report(y_test, prediction))

accuracy: 98.8%
[[4604   55]
 [  53 4268]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4659
           1       0.99      0.99      0.99      4321

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [9]:
# train with lightGBM
x_train,x_test,y_train,y_test = train_test_split(news['text'], news.target, test_size=0.2, random_state=2020)

pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LGBMClassifier())])

model = pipe.fit(x_train, y_train)
prediction = model.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))
print(confusion_matrix(y_test, prediction))
print(classification_report(y_test, prediction))

accuracy: 99.74%
[[4643   16]
 [   7 4314]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4659
           1       1.00      1.00      1.00      4321

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



### Keras Model
reference to: https://keras.io/examples/nlp/text_classification_with_transformer/

In [11]:
# import keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [12]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [13]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [14]:
# only connsider top 1000
# only consider first 200
vocab_size = 1000
maxlen = 200
x_train,x_test,y_train,y_test = train_test_split(news['text'], news.target, test_size=0.2, random_state=2020)
cvet = CountVectorizer(max_features=vocab_size).fit(x_train)
x_train = cvet.transform(x_train).toarray()
x_test = cvet.transform(x_test).toarray()
print(len(x_train), "Training sequences")
print(len(x_test), "Testing sequences")
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

35918 Training sequences
8980 Testing sequences


In [15]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [16]:
optimizer = keras.optimizers.Adam(lr=0.1)

model.compile(optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=32,epochs=5
)

Epoch 1/5
1123/1123 [==============================] - 93s 82ms/step - loss: 0.7274 - accuracy: 0.5020
Epoch 2/5
1123/1123 [==============================] - 89s 79ms/step - loss: 0.6965 - accuracy: 0.5057
Epoch 3/5
1123/1123 [==============================] - 96s 85ms/step - loss: 0.6960 - accuracy: 0.5119
Epoch 4/5
1123/1123 [==============================] - 94s 84ms/step - loss: 0.6975 - accuracy: 0.5065
Epoch 5/5
1123/1123 [==============================] - 95s 84ms/step - loss: 0.6950 - accuracy: 0.5088


### Reasona on Model Selections

Logistics Regression 
1. Easy to implement and interpret
2. Efficient to train
3. Good accuracy for binary classification

LightGBM
1. Efficient Boosting algorithm and able to train with larger dataset
2. High accuracy
3. Low memory usage

Keras Transformer
1. Can be used in parallelized calculation
2. Good training for larger dataset